In [3]:
import numpy as np
import cirq

In [8]:
class QT3:
    def __init__(self, seed=0):
        # Combinations of squares that lead to a win if they all have the same token
        self.winning_combinations = [[0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [2,4,6]]
        
        # Internal simulator
        self.simulator = cirq.Simulator()
        
        # Start a new game
        self.reset(seed)

    def reset(self, seed=0):
        # Track if a square has anything on it
        self.occupied = np.zeros(9)
        
        # Track all (independent) quantum circuits
        self.circuits = {}
        # Keep track if a square is part of a circuit yet
        self.square_to_circuit = {}
        
        # Keep track of current qutrit states
        self.distributions = np.zeros((9,3))
        # All empty
        self.distributions[:,0] = 1
        # Keep track of whether a distribution needs re-computing
        self.distribution_ready = np.zeros(9)
        
        # The representation of the board
        self.board = np.zeros(9)
        self.current_player = 'X'
        self.move_history = []
        
        # Not yet game over
        self.game_over = False
    
        # Set the seed
        np.random.seed(seed)
        
    def isEmpty(self, square):
        # Empty if full weight is in the empty state
        if( self.distributions[square][0] == 1 ):
            return True
        return False

    def isBoardFull(self):
        for s in range(9):
            if( self.isEmpty(s) ):
                return False
        return True
    
    def isClassical(self):
        # Count number of 2 qutrit circuits
        for c in self.circuits.keys():
            if len(self.circuits[c]['squares']) > 1:
                return False
        return True

    def hasWinner(self):
        winners = []
        
        print("checking winner")

        board = [np.argmax(a) for a in self.distributions]
        print(board)
        
        for c in self.winning_combinations:
            if( board[c[0]] == board[c[1]] and board[c[1]] == board[c[2]] and (board[c[0]] == 1 or board[c[0]] == 2) ):
                winners.append( board[c[0]] )

        if( len(winners) == 0 ):
            # No winnner, but board is full -> Draw
            if( self.isBoardFull() ):
                return 0

            # No winner and board not full -> Game not done yet
            return -1

        mean_winner = int(np.mean(np.array(winners)-1))
        if( mean_winner == 0.5 ):
            return 0
        elif( mean_winner < 0.5 ):
            return 1
        else:
            return 2

    def project(self):
        #Nothing to do
        if( self.isClassical() ):
            return
        
        # Run each circuit, and update the squares it runs for
        for c in self.circuits.keys():
            C = self.circuits[c]
            squares = C['squares']
            
            # 2 qubits (to make a qutrit) per square
            qubits = cirq.LineQubit.range(2*len(squares))
            circuit = cirq.Circuit()
            
            for gate in C['gates']:

                if( gate[0] == 'X' ):
                    circuit.append( cirq.X(qubits[gate[1]]) )

                if( gate[0] == 'CX' ):
                    circuit.append( cirq.SWAP(qubits[gate[1]], qubits[gate[3]])**0.5 )
                    circuit.append( cirq.SWAP(qubits[gate[2]], qubits[gate[4]])**0.5 )    

            circuit.append( cirq.measure(*qubits) )
        
            result = self.simulator.run(circuit)
            result = result.measurements[list(result.measurements.keys())[0]][0]
            
            if( len(result) == 2 ):
                if( result[0] == 1 ):
                    self.board[squares[0]] = 1
                    self.distributions[squares[0]] = [0, 1, 0]
                elif( result[1] == 1 ):
                    self.board[squares[0]] = 2
                    self.distributions[squares[0]] = [0, 0, 1]
                else:
                    self.board[squares[0]] = 0
                    self.distributions[squares[0]] = [1, 0, 0]                    
            
            if( len(result) == 4 ):
                result1 = np.array(result)[0:2]
                result2 = np.array(result)[2:4]
                
                self.board[squares[0]] = 0 if (result1[0] == 0 and result1[1] == 0) else (1 if (result1[0] == 1 and result1[1] == 0) else 2)
                self.board[squares[1]] = 0 if (result2[0] == 0 and result2[1] == 0) else (1 if (result2[0] == 1 and result2[1] == 0) else 2)
                
                X = result1[0]
                O = result1[1]
                self.distributions[squares[0]] = [1-X-O, X, O]
                X = result2[0]
                O = result2[1]
                self.distributions[squares[1]] = [1-X-O, X, O]
               
        self.circuits = {}
        self.square_to_circuit = {}

        # Update all squares (if required)
        for square in range(9):
            if( self.distributions[square][1] == 1 ):
                self.circuits[len(self.circuits)] = {'squares':square, 'gates': [('X', 0)]}
                self.square_to_circuit[square] = len(self.circuits)-1
            if( self.distributions[square][1] == 2 ):
                self.circuits[len(self.circuits)] = {'squares':square, 'gates': [('X', 1)]}
                self.square_to_circuit[square] = len(self.circuits)-1
        
    def is_legal_move(self, move):
        if( len(move) == 1 ):
            if( not self.isEmpty(move[0]) ):
                return False
            return True

        if( not self.isEmpty(move[0]) or not self.isEmpty(move[1])):
            return False

        if( move[0] == move[1] ):
            return False

        return True

    def get_legal_moves(self):
        moves = []
        # Classical moves
        for i in range(9):
            if( self.is_legal_move([i])):
                moves.append([i])

        if( self.quantum ):
            # Quantum moves
            for i in range(9):
                for j in range(i+1,9):
                    if( self.is_legal_move([i,j]) ):
                        moves.append([i,j])

        return moves
    
    def get_distribution(self, square, num_shots = 100, force_recompute=False):
        
        # Nothing to do if the circuit for this square has already been computed
        if self.distribution_ready[square] and not force_recompute:
            return
        
        # Find the circuit this square belongs to
        index = self.square_to_circuit.get(square,-1)
        
        # Fully empty
        if index == -1:
            if( self.board[square] == 0 ):
                return [1, 0, 0]
            if( self.board[square] == 1 ):
                return [0, 1, 0]
            if( self.board[square] == 2 ):
                return [0, 0, 1]
        
        # Build and sample the circuit
        C = self.circuits[index]
        squares = C['squares']
            
        # 2 qubits (to make a qutrit) per square
        qubits = cirq.LineQubit.range(2*len(squares))
        circuit = cirq.Circuit()
            
        for gate in C['gates']:

            if( gate[0] == 'X' ):
                circuit.append( cirq.X(qubits[gate[1]]) )

            if( gate[0] == 'CX' ):
                circuit.append( cirq.SWAP(qubits[gate[1]], qubits[gate[3]])**0.5 )
                circuit.append( cirq.SWAP(qubits[gate[2]], qubits[gate[4]])**0.5 )    

        circuit.append( cirq.measure(*qubits) )
        result = self.simulator.run(circuit, repetitions = num_shots) 
        result = result.measurements[list(result.measurements.keys())[0]]
        
        full_result = np.zeros((len(squares),2))
        for r in range(len(result)):
            res = result[r]
            
            for s,square in enumerate(squares):
                # Extract the results for this square
                square_result = np.array(res)[2*s:2*s+2]
                full_result[s] += np.array(square_result)
        
        full_result /= num_shots
        
        # The sites in this circuit now no longer need recomputing
        for s,square in enumerate(squares):
            self.distribution_ready[square] = 1
            X = full_result[s][0]
            O = full_result[s][1]
            E = 1-X-O
            self.distributions[square] = np.array([E,X,O])
            
        return self.distributions[square]

    def get_symbol_for_distribution(self, distribution):
        if( distribution[0] > 0.99 ):
            return '.'
        if( distribution[1] > 0.99 ):
            return 'X'
        if( distribution[2] > 0.99 ):
            return 'O'

        if( distribution[0] > 0 and distribution[1] > 0 and distribution[2] == 0 ):
            return 'x'
        if( distribution[0] > 0 and distribution[2] > 0 and distribution[1] == 0 ):
            return 'o'

    def print_board(self):
        # Sample all circuits and get prob distribution
        for i in range(3):
            line = "\t "
            for j in range(3):
                index = 3*i + j
                
                token = self.get_symbol_for_distribution( self.distributions[index] )
                line += token
                if( j != 2 ):
                    line += "  |  "
            print(line)
        
    def doClassicalMove(self, site, s):
        if( self.board[site] != 0 ):
            return False

        # For a classical move, this square must be empty. So we can safely create a new circuit
        self.circuits[len(self.circuits)] = {'squares':[site], 'gates': [('X', 0 if s == 1 else 1)] }
        self.square_to_circuit[site] = len(self.circuits)-1
        
        # This site needs re-computing
        self.distribution_ready[site] = 0
        return True

    def doQuantumMove(self, site1, site2, s):
        
        # If neither square is yet part of a circuit
        if( self.square_to_circuit.get(site1,-1) == -1 and self.square_to_circuit.get(site2,-1) == -1 ):
            
            index = len(self.circuits)
            self.circuits[index] = {'squares':[site1, site2], 'gates':[]}
            
            if s == 1:
                self.circuits[index]['gates'] = [('X', 0), ('CX', 0, 1, 2, 3)]
            if s == 2:
                self.circuits[index]['gates'] = [('X', 1), ('CX', 0, 1, 2, 3)]
            
            # Assign each of these squares to point to that circuit
            self.square_to_circuit[site1] = len(self.circuits)-1
            self.square_to_circuit[site2] = len(self.circuits)-1
            
            # These sites need recomputing
            self.distribution_ready[site1] = 0
            self.distribution_ready[site2] = 0            
            
            return True
            
        # If one of the selected squares is in a quantum pair
        elif( (self.square_to_circuit.get(site1,-1) != -1 and self.square_to_circuit.get(site2,-1) == -1) or
            (self.square_to_circuit.get(site2,-1) != -1 and self.square_to_circuit.get(site1,-1) == -1) ):

            # Find out which one was in a pair
            inPair = site1 if self.square_to_circuit.get(site1,-1) != -1 else site2
            # Get the Hilbert space
            C = self.circuits[ self.square_to_circuit[inPair] ]
            
            # Add more gates
            C['gates'] = C['gates'] + ('CX', 0, 1, 2, 3)
            
            # These sites need recomputing
            self.distribution_ready[site1] = 0
            self.distribution_ready[site2] = 0  
            
            return True
            
        return False
            
    def doMove(self, move):
        if len(move) == 1:
            site = int(move[0])
            move_success = self.doClassicalMove(site, 1 if self.current_player == 'X' else 2)
            
        if len(move) == 2:
            site1 = int(move[0])
            site2 = int(move[1])
            move_success = self.doQuantumMove(site1, site2, 1 if self.current_player == 'X' else 2)
            
        if( move_success ):
            # Add move to history
            self.move_history.append(move)
            
            # Update all squares (if required)
            for square in range(9):
                self.get_distribution(square)

            # Switch player
            self.current_player = 'X' if self.current_player == 'O' else 'O'

            # Check if full
            if( self.isBoardFull() ):
                self.project()

            # Check for winner
            self.winner = self.hasWinner()
            if( self.winner != -1 ):
                self.game_over = True

In [9]:
newGame = QT3(seed=0)

# Use a single number to place a token (e.g. '1' puts your token top-middle)
# Use two numbers to do a superposition (e.g. '08' makes a superposition between top-left and bottom-right)

while not newGame.game_over:
    print("Current player: ", newGame.current_player)
    move = input("Input your move: ")
    newGame.doMove(move)
    
    newGame.print_board()
    
if newGame.winner == 0:
    print("Draw!")
else:
    print("The winner is: ", 'X' if newGame.winner == 1 else 'O')

Current player:  X


Input your move:  01


checking winner
[1, 0, 0, 0, 0, 0, 0, 0, 0]
	 x  |  x  |  .
	 .  |  .  |  .
	 .  |  .  |  .
Current player:  O


Input your move:  23


checking winner
[1, 0, 0, 0, 0, 0, 0, 0, 0]
	 x  |  x  |  o
	 o  |  .  |  .
	 .  |  .  |  .
Current player:  X


KeyboardInterrupt: Interrupted by user